# Student Course Report Analysis
_THIS IS NOT USED, I ENDED UP DOING EVERYTHING IN JS ON THE DASHBOARD PAGE_

In [1]:
# Imports
import pandas as pd
import os
import re
import json
import matplotlib.pyplot as plt
import functools

from scipy import stats
from os.path import join, dirname
from pymongo import MongoClient
from pprint import pprint
from dotenv import load_dotenv
from IPython.display import display, JSON, HTML

# Load environment variables
load_dotenv(join(os.path.abspath(""), "../.env.local"))

pass

In [2]:
# Connect to database
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
db = mongo_client["george-beta"]

# Load data
professors = pd.DataFrame(db["professors"].find({}))
courses = pd.DataFrame(db["courses"].find({}))
sections = pd.DataFrame(db["sections"].find({}))

In [3]:
# The question text for each number
question_texts = {
    1: "My overall rating of the quality of this course is",
    2: "My overall rating of the instructor's teaching is",
    3: "The educational value of the assigned work was",
    4: "The instructor's organization of the course was",
    5: "The instructor's clarity in communicating course objectives was",
    6: "The instructor's skill in providing understandable explanations was",
    7: "The amount I learned from the course was",
    8: "The intellectual challenge presented by the course was",
    9: "The instructor's personal interest in helping students learn was",
    10: "The instructor stimulated my interest in the subject matter",
    11: "The amount of reading, homework, and other assigned work was",
    12: "The instructor was well prepared to teach class.",
    13: "The instructor encouraged students to ask questions.",
    14: "The instructor treated students with respect.",
    15: "Instructor feedback on exams/assignments was timely and helpful.",
    16: "The exams and/or evaluations were good measures of the material covered.",
    17: "My grades were determined in a fair and impartial manner.",
    18: "What grade do you think you will receive in this course?",
    19: "On average, what were the total hours spent in each 7-day week OUTSIDE of formally scheduled class time in work related to this course (including studying, reading, writing, homework, rehearsal, etc.)?"
}

# Categories of questions (subjective)
question_categories = {
    "course_quality": [1, 2, 3, 4, 5, 6],
    "course_difficulty": [7, 8, 9, 10, 11, 19],
    "instructor_quality": [12, 13, 14, 15, 16],
    "grades": [17, 18]
}

In [4]:
# Augment the data

# Assign professors to 1+ department


# Calculate the average response of every question in a category
def add_category(row, questions):
    category_sum = 0
    for question in row.report:
        if question["questionNumber"] not in questions:
            continue
        #print(question["questionNumber"])
        responses = question["responses"]
        weighted_responses = [x*(i+1) for i, x in enumerate(responses)]

        if sum(responses) == 0:
            return None
        else:
            mean_response = sum(weighted_responses) / sum(responses)

        category_sum += mean_response
        #print(responses, weighted_responses, mean_response)
    
    category_mean = category_sum / len(questions)
    
    return category_mean

for category, questions in question_categories.items():
    sections[category] = sections.apply(lambda row: add_category(row, questions), axis=1)

In [5]:
# Filter sections
def get_sections_by_department(sections, dept):
    return sections[sections.apply(lambda r: re.search(f"^{dept}", r["courseId"]) is not None, axis=1 )]

def get_sections_by_professor(sections, professor_id): 
    return sections[sections["professorId"] == professor_id]

In [6]:
random_prof = "935932"
#display(get_sections_by_professor(random_prof))